# Segmenting and Clustering Neighborhoods in Toronto
by Sebastian Spena

## 1. Import all libraries and packages neeeded to proceed

In [1]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



In [2]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import urllib.request
from bs4 import BeautifulSoup
import geocoder


print('Libraries imported.')

Libraries imported.


## 2. Download and Explore Dataset. Creating the Dataframe

__<font color=blue>Instantiate the URL using beatiful soup</font>__

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "lxml")
print('Data downloaded!')

Data downloaded!


__<font color=blue>Creating the table by scraping wikipedia</font>__

In [3]:
all_tables=soup.find_all("table")
right_table=soup.find('table', class_='wikitable sortable')
right_table
A=[]
B=[]
C=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
print('Wikipedia scraped!')

Wikipedia scraped!


__<font color=blue>Creating the dataframe and getting rid of the \n in the table</font>__

In [4]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C
df['Postal Code']=df['Postal Code'].str.replace(r"\n", "")
df['Borough']=df['Borough'].str.replace(r"\n", "")
df['Neighborhood']=df['Neighborhood'].str.replace(r"\n", "")
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


__<font color=blue>Ignoring cells with a borough that is Not Assigned</font>__

In [5]:
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.shape

(103, 3)

__<font color=blue>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough</font>__

In [7]:
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']

In [8]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [14]:
df.shape

(103, 3)

## END of first Notebook

## 3. As I have tried several times to use the GEOCODER package, but always unsuccesfully, I will load the coordinates from the csv file


In [16]:
coordinates = pd.read_csv("Geospatial_Coordinates.csv")
coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


__<font color=blue>merging the tables to append lat and lng</font>__

In [18]:
df = df.merge(coordinates, on="Postal Code", how="left")
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## END of second Notebook